In [99]:
# import libraries
import pandas as pd
import scipy.stats
import statsmodels.stats.multitest
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# project specific libs
import os
import matplotlib.pyplot as plt

In [100]:
# set working directory file path
path = '/Users/KevinBu/Desktop/clemente_lab/Projects/twinsra/code/'

In [101]:
# read in initial df
df_raw = pd.read_csv(path + 'All_counts.tsv', sep='\t')#, index_col=0)
df_raw.head()

,PID,Specimen,DeliveryMode,Live_sorted_pct,Dead_sorted_pct,Total.cells.cm2,Live.cells.cm2,Dead.cells.cm2
0,10952,Buccal_swab,Csection,98.507772,1.492228,1.568750e+07,1.491254e+07,225900.00000
1,10974,Buccal_swab,Csection,94.445613,5.554387,1.224390e+07,1.099258e+07,646478.04880
2,10021,Buccal_swab,Csection,89.707634,10.292366,5.987952e+06,5.140776e+06,589813.25300
3,10103,Buccal_swab,Vaginal,86.003100,13.996900,2.572464e+06,2.026690e+06,329841.30430
4,10114,Buccal_swab,Csection,97.649542,2.350458,3.627737e+06,3.424221e+06,82422.18978


In [102]:
# get rid of all vars not Live.cells.cm, also do log transform
# get hard copy
df_counts = df_raw.copy()

# take log
# df_counts['logLive'] = np.log(df_counts['Live.cells.cm2'])
df_counts['Total'] = df_counts['Total.cells.cm2']

# remove unnecessary columns
df_counts = df_counts.drop(['Live_sorted_pct','Dead_sorted_pct','Total.cells.cm2','Dead.cells.cm2','Live.cells.cm2'],axis=1)
df_counts.head()


,PID,Specimen,DeliveryMode,Total
0,10952,Buccal_swab,Csection,1.568750e+07
1,10974,Buccal_swab,Csection,1.224390e+07
2,10021,Buccal_swab,Csection,5.987952e+06
3,10103,Buccal_swab,Vaginal,2.572464e+06
4,10114,Buccal_swab,Csection,3.627737e+06


In [104]:
# grab subset dfs on site
df_cs = df_counts[df_counts['DeliveryMode'] == 'Csection']
df_vg = df_counts[df_counts['DeliveryMode'] == 'Vaginal']

# drop duplication?
df_cs = df_cs.drop(395) # Gauze duplication, PID 13706
df_cs = df_cs.drop(387) # Gauze duplication, PID 10805

site_to_ogdf = {
    'Csection': df_cs,
    'Vaginal': df_vg
}

site_to_newdf = {}
site_to_newdf_wide = {}

# for each site
for s in ['Csection', 'Vaginal']:
    # get relevant df
    df = site_to_ogdf[s]
    
    # drop DeliveryMode
    df = df.drop('DeliveryMode',axis=1)
    
    # get PIDs and Speciments
    # for each PID and Specimen pairing, if logLive is not a float, make it NaN
    PIDs = list(set(df['PID'].unique())) # 80 unique PIDs
    types = list(set(df['Specimen'].unique())) # 4 unique types

    for p in PIDs:
        for t in types:
            ll = df[(df['PID'] == p) & (df['Specimen'] == t)]['Total'].values # will be empty list if DNE
            if ll.size == 0:
                new_row = pd.DataFrame(data={'PID':p, 'Specimen': t, 'Total': np.nan}, index=[p]) 
                df = pd.concat([df, new_row])#, ignore_index=True)
    site_to_newdf[s] = df
    df.to_csv(path + s + '_long_df.csv')
    site_to_newdf_wide[s] = df.pivot(index='PID', columns='Specimen', values='Total')
    site_to_newdf_wide[s].to_csv(path + s + '_wide_df.csv')
    
df.head()

,PID,Specimen,Total
3,10103,Buccal_swab,2.572464e+06
6,10400,Buccal_swab,1.106411e+06
8,11022,Buccal_swab,2.510000e+07
12,11313,Buccal_swab,1.004000e+08
15,11479,Buccal_swab,5.456522e+07
